In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from tensorflow.keras.optimizers import Adam

In [6]:
from tensorflow.keras.applications import VGG19, VGG16

In [8]:
import cv2
import os 
import pickle

In [10]:
from imutils import paths

In [11]:
LABELS = set(["Crown and Root Rot", "Healthy Wheat", "Leaf Rust", "Wheat Loose Smut"])

In [18]:
count = 0 
for images in paths.list_images('Images/Crown and Root Rot/'):
    print(images)
    count = count + 1
    if count == 5:
        break

Images/Crown and Root Rot/00041.jpg
Images/Crown and Root Rot/00051.jpg
Images/Crown and Root Rot/00061.jpg
Images/Crown and Root Rot/00071.jpg
Images/Crown and Root Rot/00081.png


In [25]:
image = cv2.imread('Images/Crown and Root Rot/00041.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))

In [30]:
count = 0
for images in paths.list_images('Images/Crown and Root Rot/'):
    print(images)
    count = count + 1
    if count == 5:
        break

Images/Crown and Root Rot/00041.jpg
Images/Crown and Root Rot/00051.jpg
Images/Crown and Root Rot/00061.jpg
Images/Crown and Root Rot/00071.jpg
Images/Crown and Root Rot/00081.png


In [36]:
sample_path = list(paths.list_images('Images/Crown and Root Rot/'))[0]

In [37]:
sample_path

'Images/Crown and Root Rot/00041.jpg'

In [41]:
sample_path.split('/')

['Images', 'Crown and Root Rot', '00041.jpg']

In [42]:
os.listdir()

['.git',
 '.gitattributes',
 '.ipynb_checkpoints',
 'Images',
 'README.md',
 'Wheat Disease Detection.ipynb']

In [44]:
Labels = os.listdir('Images')

In [45]:
print(Labels)

['Crown and Root Rot', 'Healthy Wheat', 'Leaf Rust', 'Wheat Loose Smut']


In [59]:
CROWN_AND_ROOT_ROT_PATH = 'Images/Crown and Root Rot/'
HEALTHY_AND_WHEAT_PATH = 'Images/Healthy Wheat/'
LEAF_RUST_PATH = 'Images/Leaf Rust/'
WHEAT_LOOSE_SMUT_PATH = 'Images/Wheat Loose Smut/'

In [60]:
from tqdm import tqdm

In [63]:
data = []
labels = []
for image_path in tqdm(paths.list_images(CROWN_AND_ROOT_ROT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
for image_path in tqdm(paths.list_images(HEALTHY_AND_WHEAT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)

for image_path in tqdm(paths.list_images(LEAF_RUST_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
for image_path in tqdm(paths.list_images(WHEAT_LOOSE_SMUT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
    

1021it [00:03, 274.88it/s]
1146it [00:21, 53.84it/s]
1286it [01:26, 14.78it/s]
930it [00:04, 206.43it/s]


In [66]:
labels[-1]

'Wheat Loose Smut'

In [81]:
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [84]:
headmodel = VGG19(weights = "imagenet", include_top = False,
             input_tensor = Input(shape = (224, 224, 3)))
model = headmodel.output
model = AveragePooling2D(pool_size= (5, 5))(model)
model = Flatten(name = 'flatten')(model)
model = Dense(512, activation = 'relu')(model)
model = Dropout(0.2)(model)
model = Dense(len(Labels), activation = 'softmax')(model)

final_model = Model(inputs = headmodel.input, outputs = model)

for layer in headmodel.layers:
    layer.trainable = False

In [85]:
opt = Adam(lr = 1e-3)
final_model.compile(loss = "categorical_crossentropy", optimizer = opt,
                   metrics = ["accuracy"])

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [97]:
data = np.array(data)
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [98]:
labels[0]

array([1, 0, 0, 0])

In [99]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2,
                                                    stratify = labels, random_state = 101)

In [104]:
len(X_train)

3506

In [105]:
import tensorflow as tf

In [107]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [110]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14746939498463566225
]


In [103]:
final_model.fit(X_train, y_train, validation_data = (X_test, y_test), 
               epochs = 5, batch_size = 64)

Train on 3506 samples, validate on 877 samples
Epoch 1/5
3506/3506 [==============================] - 487s 139ms/sample - loss: 1.1346 - accuracy: 0.7342 - val_loss: 0.6005 - val_accuracy: 0.8153
Epoch 2/5
 256/3506 [=>............................] - ETA: 6:09 - loss: 0.5884 - accuracy: 0.8333

KeyboardInterrupt: 